In [1]:
import pandas as pd
import numpy as np 
import os
import torch
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils
import torch.nn as nn
import torch.optim as optim

## Pre-processing of data

In [3]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),
])

## Loading the Data

In [5]:
train_dir = "/Users/shreyamusini/Downloads/emotions_images/train"
test_dir = "/Users/shreyamusini/Downloads/emotions_images/test"

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
data_iter = iter(train_loader)  # Correct usage
inputs, labels = next(data_iter)
print(f"Input shape: {inputs.shape}, Label shape: {labels.shape}")

Input shape: torch.Size([32, 1, 64, 64]), Label shape: torch.Size([32])


## Convolutional Neural Network Model

In [68]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu_1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu_2 = nn.ReLU()
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)  
        self.fc1 = nn.Linear(3136, 128) 
        self.fc2 = nn.Linear(128, 7)
        self.relu_3 = nn.ReLU()
        self.relu_4 = nn.ReLU()
        
    def forward(self, x):
        x_1 = self.pool(self.relu_1(self.conv1(x))) 
        x_2 = self.pool(self.relu_2(self.conv2(x_1))) 
        x_3 = self.pool(self.relu_3(self.conv3(x_2)))
        x_4 = x_3.view(x_3.size(0), -1) 
        x_5 = self.relu_4(self.fc1(x_4))  
        x_6 = self.fc2(x_5)  
        return x_6
        

### Initialize Model

In [71]:
model_simple_cnn = SimpleCNN()

### Optimizer + Loss Function

In [74]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_simple_cnn.parameters(), lr=0.001)

## Training

In [77]:
train_losses = []
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model_simple_cnn(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_losses[-1]:.4f}')

Epoch 1/10, Loss: 1.6571
Epoch 2/10, Loss: 1.4372
Epoch 3/10, Loss: 1.3115
Epoch 4/10, Loss: 1.2105
Epoch 5/10, Loss: 1.1323
Epoch 6/10, Loss: 1.0538
Epoch 7/10, Loss: 0.9847
Epoch 8/10, Loss: 0.9142
Epoch 9/10, Loss: 0.8492
Epoch 10/10, Loss: 0.7828


## Test Accuracy

In [80]:
correct = 0
test_loss = 0
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        outputs = model_simple_cnn(images)
        test_loss += loss_func(outputs, labels).item()
        correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
print(f'Accuracy of the network on the 7200 test images: {100 * correct / len(test_loader.dataset):.2f}%, Test loss: {test_loss / len(test_loader.dataset):.4f}')

Accuracy of the network on the 7200 test images: 54.01%, Test loss: 0.0438
